## Tokens
##### Keywords: if, else, elif, while, for, in, def, return, True, False, None
##### Operators: +, -, *, /, %, **, =, ==, !=, <, >, <=, >=, and, or, not
##### Delimiters: (, ), {, }, [, ], ',' , '.', ':', ';'
##### Literals: INTEGER, FLOAT, STRING
##### Other: IDENTIFIER, INDENT, DEDENT, NEWLINE, EOF

## Token and Error Class

In [1]:
import re
from typing import Generator, List, Optional, Tuple

class Token:
    def __init__(self, type: str, value: str, line: int, column: int):
        self.type = type
        self.value = value
        self.line = line
        self.column = column
    
    def __str__(self) -> str:
        return f"Token({self.type}, '{self.value}', line={self.line}, col={self.column})"
    
    def __repr__(self) -> str:
        return self.__str__()

class Error:
    def __init__(self, message: str, line: int, column: int):
        self.message = message
        self.line = line
        self.column = column
    
    def __str__(self) -> str:
        return f"Error: {self.message} at line {self.line}, column {self.column}"


## Lexer

In [2]:
class Lexer:
    # Defining regular expressions for token
    TOKEN_SPECS = [
        ('COMMENT', r'#.*'),                                  # Comments
        ('STRING', r'\"([^\\\"]|\\.)*\"|\'([^\\\']|\\.)*\''), # String literals
        ('FLOAT', r'\d+\.\d+'),                               # Float literals
        ('INTEGER', r'\d+'),                                  # Integer literals
        ('KEYWORD', r'(if|else|elif|while|for|in|def|return|True|False|None)\b'),  # Keywords
        ('IDENTIFIER', r'[a-zA-Z_]\w*'),                      # Identifiers
        # Operators (multi-character ones first)
        ('OP_EQ', r'=='),
        ('OP_NE', r'!='),
        ('OP_LE', r'<='),
        ('OP_GE', r'>='),
        ('OP_ASSIGN', r'='),
        ('OP_PLUS', r'\+'),
        ('OP_MINUS', r'-'),
        ('OP_MULT', r'\*\*|\/\/|\*|\/'),  # ** (power), // (floor div), * (mult), / (div)
        ('OP_MOD', r'%'),
        ('OP_LT', r'<'),
        ('OP_GT', r'>'),
        # Delimiters
        ('LPAREN', r'\('),
        ('RPAREN', r'\)'),
        ('LBRACKET', r'\['),
        ('RBRACKET', r'\]'),
        ('LBRACE', r'\{'),
        ('RBRACE', r'\}'),
        ('COMMA', r','),
        ('DOT', r'\.'),
        ('COLON', r':'),
        ('SEMICOLON', r';'),
        ('NEWLINE', r'\n'),
        ('WHITESPACE', r'[ \t]+'),                            # Whitespace
        # INDENT and DEDENT tokens are not matched by regex but generated based on whitespace
    ]
    
    def __init__(self, source_code: str):
        self.source_code = source_code
        self.tokens = []
        self.errors = []
        self.line = 1
        self.column = 1
        self.indent_levels = [0]  # Start with indent level 0
    
    def tokenize(self) -> Generator[Token, None, None]:
        """Tokenize the source code and yield tokens."""
        # Process the source code line by line to handle indentation properly
        lines = self.source_code.split('\n')
        
        for line_num, line in enumerate(lines):
            self.line = line_num + 1
            self.column = 1
            
            #calculating the amount of leading whitespace (indentation) at the beginning of the current line
            if line.strip():  # Non-empty line      #strip() is equivalent of trim() in JS
                indent_size = len(line) - len(line.lstrip())    #lstrip() will remove leading spaces in the line i.e. remove any indentation; and the difference in length tells you how much whitespace there was
                indent_tokens = self._handle_indentation(indent_size)
                for token in indent_tokens:
                    yield token
            else:   # Skip empty lines but still count them for line numbers
                yield Token('NEWLINE', '\\n', self.line, self.column)
                continue
            
            # Process the rest of the line
            i = indent_size if 'indent_size' in locals() else 0
            line_content = line
            
            while i < len(line_content):    #traversing a line character by character
                match = None
                
                ## Skip spaces and tabs (already handled indentation)
                ##if line_content[i].isspace():   #isspace() checks if the entire line is whitespace or not; here it checks whether a single character is whitespace or not since its applied on line_content[i]
                ##    i += 1
                ##    self.column += 1
                ##    continue
                
                # Try to match each token pattern
                for token_type, pattern in self.TOKEN_SPECS:
                    regex = re.compile(pattern) #compiling the pattern into a regex so that we can use it to find tokens in the line
                    match = regex.match(line_content, i)    #match() will check if the "starting" of the string matches the given regex or not, line_content gives the line whose starting it has to compare with the regex with i telling from which point in the line to assume as thhe starting of the line
                    #if a match is found, 'match' will contain an object with info about the found match (like start and end, its value and so on), otherwise it'll contain "none"
                    
                    if match:
                        value = match.group(0)  #group will give the value of the match object
                        
                        if token_type == 'WHITESPACE':
                            #whitespace is not yielded becuz parser receives stream of tokens without space and doesnt care about space
                            # Just update column and continue
                            self.column += len(value)
                            i += len(value)
                            continue
                        elif token_type == 'COMMENT':
                            # Ignore comments
                            i += len(value)
                            self.column += len(value)
                            break   # Break out of the current character processing loop after handling comments
                        else:
                            # For all other tokens
                            token = Token(token_type, value, self.line, self.column)
                            yield token
                        
                        i += len(value)
                        self.column += len(value)
                        break
                
                if not match:
                    # No token matched, raise an error
                    error_msg = f"Invalid character: '{line_content[i]}'"
                    error = Error(error_msg, self.line, self.column)
                    self.errors.append(error)
                    print(error)  # Print the error but continue
                    i += 1
                    self.column += 1
            
            # Add NEWLINE at the end of each line except the last one
            if line_num < len(lines) - 1 or self.source_code.endswith('\n'):
                yield Token('NEWLINE', '\\n', self.line, self.column)
        
        # Output any pending dedents at the end of the file
        #end of file means all indented blocks have been dedented so pop all elements of the indent_levels stack
        while len(self.indent_levels) > 1:
            self.indent_levels.pop()    
            yield Token('DEDENT', '', self.line, self.column)
        
        # End of file token
        yield Token('EOF', '', self.line, self.column)
    
    def _handle_indentation(self, indent_size: int) -> List[Token]:
        """Handle Python's indentation-based block structure. Returns a list of INDENT or DEDENT tokens as needed."""
        tokens = []
        previous_line_indent = self.indent_levels[-1]
        
        if indent_size > previous_line_indent:
            # This is an indentation (start of a new block)
            self.indent_levels.append(indent_size)  #when a new block is recognized through indentation its indentation level is pushed in the stack so that w ecan later check whether that block was dedented properly or not
            tokens.append(Token('INDENT', ' ' * (indent_size - previous_line_indent), self.line, 1))
        
        elif indent_size < previous_line_indent:
            # This is a dedentation (end of one or more blocks)
            while self.indent_levels and indent_size < self.indent_levels[-1]: # ensuring that the latest code block is being dedented
                self.indent_levels.pop()
                tokens.append(Token('DEDENT', '', self.line, 1))
            
            if indent_size != self.indent_levels[-1]:
                # Invalid indentation
                error_msg = f"Inconsistent indentation"
                error = Error(error_msg, self.line, 1)
                self.errors.append(error)
                print(error)  # Print the error but continue
                
                
                # and what about handling indent_size == self.indent_levels[-1]
        return tokens

## Test Script for Lexer

In [3]:
def test_lexer(source_code, test_name=""):
    print(f"\n=== Testing {test_name} ===")
    print(f"Source code:\n{source_code}")
    print("\nTokens:")
    
    lexer = Lexer(source_code)
    token_count = 0
    
    try:
        for token in lexer.tokenize():
            print(token)
            token_count += 1
        
        print(f"\nTotal tokens: {token_count}")
        if lexer.errors:
            print(f"\nErrors ({len(lexer.errors)}):")
            for error in lexer.errors:
                print(f"  {error}")
    except Exception as e:
        print(f"Exception occurred: {str(e)}")
        import traceback
        traceback.print_exc()

# Test cases
if __name__ == "__main__":
    # Test 1: Basic variable assignment
    test_lexer("x = 10", "Basic Assignment")

    # Test 2: Simple function definition
    test_lexer("""def add(a, b):
    return a + b""", "Function Definition")

    # Test 3: If statement with proper indentation
    test_lexer("""if x > 10:
    print("Greater than 10")
else:
    print("Less than or equal to 10")""", "If Statement")

    # Test 4: Various literals and operators
    test_lexer("""# Testing literals
x = 42
y = 3.14
name = "John"
flag = True
result = x + y * 2
equal = x == y""", "Literals and Operators")

    # Test 5: Loops
    test_lexer("""# Testing loops
for i in range(10):
    if i % 2 == 0:
        print("Even")
    else:
        print("Odd")

# While loop
count = 0
while count < 5:
    count += 1
""", "Loops")

    # Test 6: Test error handling
    test_lexer("x = 10 @", "Error Handling")

    # Test 7: Test inconsistent indentation
    test_lexer("""def test():
    x = 1
   y = 2  # Inconsistent indentation
""", "Inconsistent Indentation")

    # Interactive testing option
    do_interactive = input("\nDo you want to run an interactive test? (y/n): ")
    if do_interactive.lower() == 'y':
        print("\n=== Interactive Lexer Test ===")
        print("Enter Python code (type 'exit()' on a new line to finish):")
        
        lines = []
        while True:
            line = input("> ")
            if line.strip() == "exit()":
                break
            lines.append(line)
        
        source_code = "\n".join(lines)
        test_lexer(source_code, "Interactive Input")



=== Testing Basic Assignment ===
Source code:
x = 10

Tokens:
Token(IDENTIFIER, 'x', line=1, col=1)
Token(OP_ASSIGN, '=', line=1, col=3)
Token(INTEGER, '10', line=1, col=5)
Token(EOF, '', line=1, col=7)

Total tokens: 4

=== Testing Function Definition ===
Source code:
def add(a, b):
    return a + b

Tokens:
Token(KEYWORD, 'def', line=1, col=1)
Token(IDENTIFIER, 'add', line=1, col=5)
Token(LPAREN, '(', line=1, col=8)
Token(IDENTIFIER, 'a', line=1, col=9)
Token(COMMA, ',', line=1, col=10)
Token(IDENTIFIER, 'b', line=1, col=12)
Token(RPAREN, ')', line=1, col=13)
Token(COLON, ':', line=1, col=14)
Token(NEWLINE, '\n', line=1, col=15)
Token(INDENT, '    ', line=2, col=1)
Token(KEYWORD, 'return', line=2, col=1)
Token(IDENTIFIER, 'a', line=2, col=8)
Token(OP_PLUS, '+', line=2, col=10)
Token(IDENTIFIER, 'b', line=2, col=12)
Token(DEDENT, '', line=2, col=13)
Token(EOF, '', line=2, col=13)

Total tokens: 16

=== Testing If Statement ===
Source code:
if x > 10:
    print("Greater than 10")
else

## ***ABSTRACT SYNTAX TREE***

In [4]:
#This file is of abstract syntax tree (AST) node definitions.

from dataclasses import dataclass
from typing import List, Optional, Union

# Base class for all AST nodes ASTNode is the base (parent) class for all other AST node types.
#It doesn’t do anything itself, but helps us group all node types together under one type.

class ASTNode:
    pass

# Root node; body is a list of things in the program like function definitions, statements, etc.
@dataclass
class Program(ASTNode):
    body: List[ASTNode]
    line: int
    column: int

# Statements
@dataclass
class FunctionDef(ASTNode):
    name: str
    params: List[str]
    body: List[ASTNode]
    line: int
    column: int

@dataclass
class IfStatement(ASTNode):
    condition: ASTNode
    then_branch: List[ASTNode]
    elif_branches: List[tuple[ASTNode, List[ASTNode]]]
    else_branch: Optional[List[ASTNode]]
    line: int
    column: int

@dataclass
class WhileLoop(ASTNode):
    condition: ASTNode
    body: List[ASTNode]
    line: int
    column: int

@dataclass
class ForLoop(ASTNode):
    var: str
    iterable: ASTNode
    body: List[ASTNode]
    line: int
    column: int

@dataclass
class Return(ASTNode):
    value: Optional[ASTNode]
    line: int
    column: int

@dataclass
class Assignment(ASTNode):
    target: str
    value: ASTNode
    line: int
    column: int

@dataclass
class ExpressionStatement(ASTNode):
    expression: ASTNode
    line: int
    column: int

# Expressions
@dataclass
class BinaryOp(ASTNode):
    left: ASTNode
    operator: str
    right: ASTNode
    line: int
    column: int

@dataclass
class UnaryOp(ASTNode):
    operator: str
    operand: ASTNode
    line: int
    column: int

@dataclass
class Call(ASTNode):
    func: ASTNode
    args: List[ASTNode]
    line: int
    column: int

@dataclass
class Identifier(ASTNode):
    name: str
    line: int
    column: int

@dataclass
class Literal(ASTNode):
    value: Union[str, int, float, bool, None]
    line: int
    column: int


## ***AST VISUALIZER***

In [ ]:
# ast_visualizer.py

from graphviz import Digraph
from typing import Union, List
def print_ast(node: ASTNode, indent: str = ''):
    info = []
    if hasattr(node, "inferred_type") and node.inferred_type:
        info.append(f"type={node.inferred_type}")
    if hasattr(node, "constant_value") and node.constant_value is not None:
        info.append(f"const={node.constant_value}")
    info_str = f" [{', '.join(info)}]" if info else ""

    if isinstance(node, Program):
        print(indent + "Program" + info_str)
        for stmt in node.body:
            print_ast(stmt, indent + "  ")
    elif isinstance(node, FunctionDef):
        info = []
        if hasattr(node, "return_type") and node.return_type:
            info.append(f"return_type={node.return_type}")
        info_str = f" [{', '.join(info)}]" if info else ""
        print(indent + f"FunctionDef({node.name})" + info_str)
        for stmt in node.body:
            print_ast(stmt, indent + "  ")
    elif isinstance(node, IfStatement):
        print(indent + "IfStatement" + info_str)
        print(indent + "  Condition:")
        print_ast(node.condition, indent + "    ")
        print(indent + "  Then:")
        for stmt in node.then_branch:
            print_ast(stmt, indent + "    ")
        for cond, branch in node.elif_branches:
            print(indent + "  Elif:")
            print_ast(cond, indent + "    ")
            for stmt in branch:
                print_ast(stmt, indent + "      ")
        if node.else_branch:
            print(indent + "  Else:")
            for stmt in node.else_branch:
                print_ast(stmt, indent + "    ")
    elif isinstance(node, WhileLoop):
        print(indent + "WhileLoop" + info_str)
        print(indent + "  Condition:")
        print_ast(node.condition, indent + "    ")
        print(indent + "  Body:")
        for stmt in node.body:
            print_ast(stmt, indent + "    ")
    elif isinstance(node, ForLoop):
        print(indent + f"ForLoop(var={node.var})" + info_str)
        print(indent + "  Iterable:")
        print_ast(node.iterable, indent + "    ")
        print(indent + "  Body:")
        for stmt in node.body:
            print_ast(stmt, indent + "    ")
    elif isinstance(node, Return):
        print(indent + "Return" + info_str)
        if node.value:
            print_ast(node.value, indent + "  ")
    elif isinstance(node, Assignment):
        print(indent + f"Assignment(target={node.target})" + info_str)
        print_ast(node.value, indent + "  ")
    elif isinstance(node, ExpressionStatement):
        print(indent + "ExpressionStatement" + info_str)
        print_ast(node.expression, indent + "  ")
    elif isinstance(node, BinaryOp):
        print(indent + f"BinaryOp({node.operator})" + info_str)
        print_ast(node.left, indent + "  ")
        print_ast(node.right, indent + "  ")
    elif isinstance(node, UnaryOp):
        print(indent + f"UnaryOp({node.operator})" + info_str)
        print_ast(node.operand, indent + "  ")
    elif isinstance(node, Call):
        print(indent + "Call" + info_str)
        print_ast(node.func, indent + "  ")
        for arg in node.args:
            print_ast(arg, indent + "    ")
    elif isinstance(node, Identifier):
        print(indent + f"Identifier({node.name})" + info_str)
    elif isinstance(node, Literal):
        print(indent + f"Literal({repr(node.value)})" + info_str)
    else:
        print(indent + f"UnknownNode({node})" + info_str)
        
# Optional: Graphviz visualizer
def _build_graph(node: ASTNode, dot: Digraph, parent: str = None, counter=[0]):
    node_id = f"node{counter[0]}"
    counter[0] += 1

    # Build label with annotations
    label = type(node).__name__
    # Add main info
    if isinstance(node, Identifier):
        label += f"({node.name})"
    elif isinstance(node, Literal):
        label += f"({repr(node.value)})"
    elif isinstance(node, Assignment):
        label += f"({node.target})"
    elif isinstance(node, BinaryOp):
        label += f"({node.operator})"
    elif isinstance(node, UnaryOp):
        label += f"({node.operator})"
    elif isinstance(node, FunctionDef):
        label += f"({node.name})"
        if hasattr(node, "return_type") and node.return_type:
            label += f"\\nreturn_type={node.return_type}"
    elif isinstance(node, ForLoop):
        label += f"({node.var})"

    # Add annotation info
    info = []
    if hasattr(node, "inferred_type") and node.inferred_type:
        info.append(f"type={node.inferred_type}")
    if hasattr(node, "constant_value") and node.constant_value is not None:
        info.append(f"const={node.constant_value}")
    if info:
        label += "\\n" + ", ".join(info)

    dot.node(node_id, label)

    if parent:
        dot.edge(parent, node_id)

    for field in getattr(node, '__dataclass_fields__', {}):
        child = getattr(node, field)
        if isinstance(child, ASTNode):
            _build_graph(child, dot, node_id, counter)
        elif isinstance(child, list):
            for item in child:
                if isinstance(item, ASTNode):
                    _build_graph(item, dot, node_id, counter)
        elif isinstance(child, tuple):
            for item in child:
                if isinstance(item, ASTNode):
                    _build_graph(item, dot, node_id, counter)

    return dot

def visualize_ast(node: ASTNode):
    dot = Digraph(comment="AST")
    _build_graph(node, dot)
    return dot

## ***PARSER***

In [6]:
# Custom error class for parser errors
class ParserError(Exception):
    pass

# Parser class converts tokens into an AST
class Parser:
    def __init__(self, lexer: Lexer):
        self.tokens = list(lexer.tokenize())  # Convert lexer generator into list of tokens
        self.pos = 0  # Current position in token list
        self.current_token = self.tokens[self.pos]  # Currently looked-at token

    def error(self, msg: str):
        # Raise a ParserError with current token's position
        raise ParserError(f"{msg} at line {self.current_token.line}, column {self.current_token.column}")
    
    def skip_newlines(self):
        # Ignore NEWLINE tokens before/after blocks or statements
        while self.current_token.type == 'NEWLINE':
            self.advance()

    def advance(self):
        # Move to the next token
        self.pos += 1
        if self.pos < len(self.tokens):
            self.current_token = self.tokens[self.pos]

    def expect(self, token_type: str):
        # Ensure the current token is of the expected type, or throw error
        if self.current_token.type == token_type:
            self.advance()
        else:
            self.error(f"Expected token {token_type}, got {self.current_token.type}")

    def match(self, token_type: str):
        # Match a token and advance if matched
        if self.current_token.type == token_type:
            self.advance()
            return True
        return False

    def parse(self) -> Program:
        # Parse a full program (list of statements)
        body = []
        self.skip_newlines()
        while self.current_token.type != 'EOF':
            stmt = self.parse_statement()
            if stmt:
                body.append(stmt)
            self.skip_newlines()
        return Program(body=body, line=0, column=0) 

    def parse_statement(self) -> ASTNode:
        # Decide which type of statement to parse
        if self.current_token.type == 'KEYWORD':
            if self.current_token.value == 'def':
                return self.parse_function_def()
            elif self.current_token.value == 'return':
                return self.parse_return()
            elif self.current_token.value == 'if':
                return self.parse_if()
            elif self.current_token.value == 'while':
                return self.parse_while()
            elif self.current_token.value == 'for':
                return self.parse_for()
            
        #######COULDNT WORK DURING SEMANTIC ANALYSIS SO CHNAGED THE LOGIC HERE##########
        # If not a keyword, it's either an assignment or expression
        # expr = self.parse_expression()
        # if isinstance(expr, Identifier) and self.match('OP_ASSIGN'):
        #     value = self.parse_expression()
        #     return Assignment(target=expr.name, value=value)
        # return ExpressionStatement(expr)
        ################################################################################
        
        # If not a keyword, it could be assignment or expression
        if self.current_token.type == 'IDENTIFIER':
            # Lookahead to check for assignment
            next_token = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
            if next_token and next_token.type == 'OP_ASSIGN':
                assign_token = self.current_token
                identifier = self.current_token.value
                self.advance()  # consume identifier
                self.advance()  # consume '='
                value = self.parse_expression()
                return Assignment(target=identifier, value=value, line=assign_token.line, column=assign_token.column)

        # Otherwise, treat it as an expression statement
        expr_token = self.current_token
        expr = self.parse_expression()
        return ExpressionStatement(expression=expr, line=expr_token.line, column=expr_token.column)


    def parse_function_def(self) -> FunctionDef:
        # Parse function definition: def name(params):\n  <body>
        def_token = self.current_token
        self.expect('KEYWORD')  # 'def'
        if self.current_token.type != 'IDENTIFIER':
            self.error("Expected function name")
        name = self.current_token.value
        self.advance()
        self.expect('LPAREN')

        # Parse parameter list
        params = []
        if self.current_token.type != 'RPAREN':
            while True:
                if self.current_token.type != 'IDENTIFIER':
                    self.error("Expected parameter name")
                params.append(self.current_token.value)
                self.advance()
                if not self.match('COMMA'):
                    break

        self.expect('RPAREN')
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        body = self.parse_block()
        return FunctionDef(name=name, params=params, body=body, line=def_token.line, column=def_token.column)

    def parse_return(self) -> Return:
        token = self.current_token
        # Parse return statement
        self.expect('KEYWORD')  # 'return'
        if self.current_token.type == 'NEWLINE':
            return Return(value=None, line=token.line, column=token.column)
        value = self.parse_expression()
        return Return(value=value, line=token.line, column=token.column)

    def parse_if(self) -> IfStatement:
        if_token = self.current_token
        # Parse if-elif-else statement
        self.expect('KEYWORD')  # 'if'
        condition = self.parse_expression()
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        then_branch = self.parse_block()

        elif_branches = []
        # Handle optional elif blocks
        while self.current_token.type == 'KEYWORD' and self.current_token.value == 'elif':
            self.advance()
            cond = self.parse_expression()
            self.expect('COLON')
            self.expect('NEWLINE')
            self.expect('INDENT')
            body = self.parse_block()
            elif_branches.append((cond, body))

        else_branch = None
        # Handle optional else block
        if self.current_token.type == 'KEYWORD' and self.current_token.value == 'else':
            self.advance()
            self.expect('COLON')
            self.expect('NEWLINE')
            self.expect('INDENT')
            else_branch = self.parse_block()

        return IfStatement(condition, then_branch, elif_branches, else_branch, line=if_token.line, column=if_token.column)

    def parse_while(self) -> WhileLoop:
        while_token = self.current_token
        # Parse while loop
        self.expect('KEYWORD')  # 'while'
        condition = self.parse_expression()
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        body = self.parse_block()
        return WhileLoop(condition=condition, body=body, line=while_token.line, column=while_token.column)

    def parse_for(self) -> ForLoop:
        for_token = self.current_token
        # Parse for loop: for x in iterable:
        self.expect('KEYWORD')  # 'for'
        if self.current_token.type != 'IDENTIFIER':
            self.error("Expected variable name")
        var = self.current_token.value
        self.advance()
        self.expect('KEYWORD')  # 'in'
        iterable = self.parse_expression()
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        body = self.parse_block()
        return ForLoop(var=var, iterable=iterable, body=body, line=for_token.line, column=for_token.column)

    def parse_block(self) -> List[ASTNode]:
        # Parse an indented block of statements
        body = []
        self.skip_newlines()
        while self.current_token.type not in ('DEDENT', 'EOF'):
            stmt = self.parse_statement()
            if stmt:
                body.append(stmt)
            self.skip_newlines()
        self.expect('DEDENT')
        return body

    def parse_expression(self, precedence=0) -> ASTNode:
        # Parse binary expressions with precedence (e.g., 1 + 2 * 3)
        left = self.parse_primary()
        while self.is_operator(self.current_token) and self.get_precedence(self.current_token) >= precedence:
            op_token = self.current_token
            self.advance()
            right = self.parse_expression(self.get_precedence(op_token) + 1)
            left = BinaryOp(left=left, operator=op_token.value, right=right, line=op_token.line, column=op_token.column)
        return left

    def parse_primary(self) -> ASTNode:
        # Parse literals, variables, function calls, unary ops, and parenthesis
        token = self.current_token
        if token.type == 'INTEGER':
            self.advance()
            return Literal(value=int(token.value), line=token.line, column=token.column)
        elif token.type == 'FLOAT':
            self.advance()
            return Literal(value=float(token.value), line=token.line, column=token.column)
        elif token.type == 'STRING':
            self.advance()
            return Literal(value=token.value[1:-1], line=token.line, column=token.column)
        elif token.type == 'KEYWORD':
            if token.value in ('True', 'False', 'None'):
                self.advance()
                val = {'True': True, 'False': False, 'None': None}[token.value]
                return Literal(value=val, line=token.line, column=token.column)
        elif token.type == 'IDENTIFIER':
            self.advance()
            if self.match('LPAREN'):  # Function call
                args = []
                if self.current_token.type != 'RPAREN':
                    while True:
                        args.append(self.parse_expression())
                        if not self.match('COMMA'):
                            break
                self.expect('RPAREN')
                return Call(func=Identifier(name=token.value, line=token.line, column=token.column), args=args, line=token.line, column=token.column)
            return Identifier(name=token.value, line=token.line, column=token.column)
        elif token.type == 'OP_MINUS':
            self.advance()
            operand = self.parse_primary()
            return UnaryOp(operator='-', operand=operand)
        elif token.type == 'LPAREN':
            self.advance()
            expr = self.parse_expression()
            self.expect('RPAREN')
            return expr
        else:
            self.error(f"Unexpected token {token}")

    def is_operator(self, token: Token) -> bool:
        # Check if token is an operator
        return token.type.startswith('OP_')

    def get_precedence(self, token: Token) -> int:
        # Define operator precedence levels (higher = tighter binding)
        precedence = {
            'OP_OR': 1,
            'OP_AND': 2,
            'OP_EQ': 3, 'OP_NE': 3,
            'OP_LT': 4, 'OP_LE': 4, 'OP_GT': 4, 'OP_GE': 4,
            'OP_PLUS': 5, 'OP_MINUS': 5,
            'OP_MULT': 6, 'OP_MOD': 6,
            'OP_ASSIGN': 0,  # Assignment is handled separately
        }
        return precedence.get(token.type, -1)


## Testing upto parser and AST

In [11]:

# code = '''
# def add(x, y):
#     return x + y

# a = add(3, 5)
# '''

# code = """
# def foo(x):
#     if x > 0:
#         return x
#     else:
#         return -x
# """

code = '''
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

def is_even(num):
    return num % 2 == 0

x = 5
fact = factorial(x)

if is_even(fact):
    print("Even factorial")
else:
    print("Odd factorial")

for i in range(3):
    result = factorial(i)
    print(result)
'''


lexer = Lexer(code)
parser = Parser(lexer)
ast = parser.parse()

#Console view
print_ast(ast)


#graph
dot = visualize_ast(ast)
dot.render('ast_tree', view=True, format='png')




Program
  FunctionDef(factorial)
    IfStatement
      Condition:
        BinaryOp(==)
          Identifier(n)
          Literal(0)
      Then:
        Return
          Literal(1)
      Else:
        Return
          BinaryOp(*)
            Identifier(n)
            Call
              Identifier(factorial)
                BinaryOp(-)
                  Identifier(n)
                  Literal(1)
  FunctionDef(is_even)
    Return
      BinaryOp(==)
        BinaryOp(%)
          Identifier(num)
          Literal(2)
        Literal(0)
  Assignment(target=x)
    Literal(5)
  Assignment(target=fact)
    Call
      Identifier(factorial)
        Identifier(x)
  IfStatement
    Condition:
      Call
        Identifier(is_even)
          Identifier(fact)
    Then:
      ExpressionStatement
        Call
          Identifier(print)
            Literal('Even factorial')
    Else:
      ExpressionStatement
        Call
          Identifier(print)
            Literal('Odd factorial')
  ForLoop(var=i)


'ast_tree.png'

## ***SEMANTIC ANALYSIS***

In [ ]:
# Custom exception class used to indicate semantic errors in the code being analyzed
class SemanticError(Exception):
    pass

# Set of built-in function names that user-defined functions/variables should not override
BUILTINS = {'print', 'len', 'range', 'input', 'int', 'float', 'str', 'bool'}  # Extend as needed

# Scope class manages variables/functions within a block or function, supports nested scoping
class Scope:
    def __init__(self, parent=None):
        self.parent = parent             # Reference to the parent scope (for nested scopes)
        self.variables = {}              # declared variable names in current scope
        self.functions = {}              # Dictionary of declared functions: {func_name: param_list}
        self.in_function = False         # Flag to check if this scope is inside a function

    # Declare a variable in the current scope; return False if it already exists
    def declare_var(self, name, var_type=None):
        if name in self.variables:
            return False  # Duplicate in same scope
        self.variables[name] = {"type": var_type}
        return True

    # Check whether a variable is declared in current or any parent scope
    def is_var_declared(self, name):
        if name in self.variables:
            return True
        if self.parent:
            return self.parent.is_var_declared(name)
        return False

    # Declare a function with a list of parameters; returns False if already declared
    def declare_func(self, name, func_def):
        if name in self.functions:
            return False
        self.functions[name] = func_def
        return True

    # Recursively retrieve the parameter list of a declared function
    def get_func(self, name):
        if name in self.functions:
            return self.functions[name]
        if self.parent:
            return self.parent.get_func(name)
        return None

    # Check if a function is declared by trying to get it
    def is_func_declared(self, name):
        return self.get_func(name) is not None
    
    #to show symbol table 
    def print_symbols(self, indent=0):
        prefix = "  " * indent
        print(f"{prefix}Scope:")
        if self.variables:
            for name, info in self.variables.items():
                print(f"{prefix}  Variable: {name}, Type: {info.get('type', 'unknown')}")
        if self.functions:
            for fname, func_def in self.functions.items():
                param_strs = [f"{p}: {t}" for p, t in func_def.param_types.items()]
                return_type = getattr(func_def, 'return_type', None)
                return_type_str = f", Return: {return_type}" if return_type else ""
                print(f"{prefix}  Function: {fname}({', '.join(param_strs)}){return_type_str}")
        if self.parent:
            self.parent.print_symbols(indent + 1)

# SemanticAnalyzer walks through the AST to validate semantics and detect errors
class SemanticAnalyzer:
    def __init__(self):
        self.global_scope = Scope()        # Top-level scope for global variables/functions
        self.current_scope = self.global_scope  # Pointer to current active scope
        self.errors = []                   # List to store error messages


    # Report a semantic error with line, column, and node context
    def error(self, msg, node):
        line = getattr(node, 'line', '?')
        col = getattr(node, 'column', '?')
        node_name = getattr(node, 'name', type(node).__name__)
        self.errors.append(f"[Line {line}, Column {col}] Error at '{node_name}': {msg}")
        # self.errors.append(formatted)

    # Entry point to start analyzing the AST
    def analyze(self, node):
        self.visit(node)  # Start visiting from the root node
        if self.errors:
            raise SemanticError("\n".join(self.errors))

    # Dispatch method that calls the appropriate visit_* method based on node type
    def visit(self, node):
        method = 'visit_' + type(node).__name__
        visitor = getattr(self, method, self.generic_visit)
        return visitor(node)

    # Generic visitor for composite nodes (e.g., containing other nodes in fields/lists/tuples)
    def generic_visit(self, node):
        for field in getattr(node, '__dataclass_fields__', {}):
            value = getattr(node, field)
            if isinstance(value, ASTNode):
                self.visit(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, ASTNode):
                        self.visit(item)
            elif isinstance(value, tuple):
                for item in value:
                    if isinstance(item, ASTNode):
                        self.visit(item)

    # Visit a program node which consists of multiple statements
    def visit_Program(self, node: Program):
        for stmt in node.body:
            self.visit(stmt)

    # Visit a function definition and validate function name, parameters, and body
    def visit_FunctionDef(self, node: FunctionDef):
        if not self.current_scope.declare_func(node.name, node):
            self.error(f"Duplicate function definition '{node.name}'", node)
        func_scope = Scope(parent=self.current_scope)
        func_scope.in_function = True

        node.param_types = {}
        for param in node.params:
            p_name = param if isinstance(param, str) else param[0]
            p_type = None if isinstance(param, str) else param[1]
            func_scope.declare_var(p_name, p_type)
            node.param_types[p_name] = p_type

        prev_scope = self.current_scope
        self.current_scope = func_scope

        return_types = set()
        for stmt in node.body:
            self.visit(stmt)
            if isinstance(stmt, Return) and hasattr(stmt, 'inferred_type') and stmt.inferred_type is not None:
                return_types.add(stmt.inferred_type)

        self.current_scope = prev_scope
        node.return_type = return_types.pop() if len(return_types) == 1 else 'multiple' if return_types else 'None'

    # Visit an assignment: validate left-hand name, check for name conflicts, and analyze RHS
    def visit_Assignment(self, node: Assignment):
        if node.target in BUILTINS:
            self.error(f"Cannot assign to built-in name '{node.target}'", node)
        if self.current_scope.is_func_declared(node.target):
            self.error(f"Cannot assign to function name '{node.target}'", node)
        self.visit(node.value)
        var_type = getattr(node.value, 'inferred_type', None)
        node.inferred_type = var_type
        self.current_scope.declare_var(node.target, var_type)

    # Visit an identifier node to check if it was declared
    def visit_Identifier(self, node: Identifier):
        scope = self.current_scope
        while scope:
            if node.name in scope.variables:
                node.inferred_type = scope.variables[node.name]["type"]
                return
            scope = scope.parent
        if self.current_scope.is_func_declared(node.name):
            self.error(f"Function '{node.name}' used as a variable", node)
        else:
            self.error(f"Undeclared variable '{node.name}'", node)

    # Visit a function call and validate function existence and argument count
    def visit_Call(self, node: Call):
        func_def = self.current_scope.get_func(node.func.name)
        if func_def is None:
            self.error(f"Call to undefined function '{node.func.name}'", node)
        else:
            # Infer parameter types if not set
            param_types_changed = False
            for (pname, exp_type), arg in zip(func_def.param_types.items(), node.args):
                self.visit(arg)
                actual = getattr(arg, 'inferred_type', None)
                if exp_type is None:
                    func_def.param_types[pname] = actual  # Infer type from first call
                    param_types_changed = True
                elif actual != exp_type:
                    self.error(f"Type mismatch in argument: expected {exp_type}, got {actual}", arg)
            if len(node.args) != len(func_def.param_types):
                self.error(f"Function '{node.func.name}' expects {len(func_def.param_types)} arguments, got {len(node.args)}", node)
            # Re-analyze function body if parameter types were updated
            if param_types_changed:
                prev_scope = self.current_scope
                func_scope = Scope(parent=self.global_scope)
                func_scope.in_function = True
                for pname, ptype in func_def.param_types.items():
                    func_scope.declare_var(pname, ptype)
                self.current_scope = func_scope
                return_types = set()
                for stmt in func_def.body:
                    self.visit(stmt)
                    if isinstance(stmt, Return) and hasattr(stmt, 'inferred_type') and stmt.inferred_type is not None:
                        return_types.add(stmt.inferred_type)
                self.current_scope = prev_scope
                func_def.return_type = return_types.pop() if len(return_types) == 1 else 'multiple' if return_types else 'None'
        if func_def and hasattr(func_def, 'return_type'):
            node.inferred_type = func_def.return_type


    # Visit a return statement and ensure it's inside a function
    def visit_Return(self, node: Return):
        scope = self.current_scope
        while scope:
            if scope.in_function:
                break
            scope = scope.parent
        else:
            self.error("Return statement outside function", node)

        if node.value:
            self.visit(node.value)
            node.inferred_type = getattr(node.value, 'inferred_type', None)

    # Visit if/elif/else blocks and analyze each conditional branch
    def visit_IfStatement(self, node: IfStatement):
        self.visit(node.condition)
        for stmt in node.then_branch:
            self.visit(stmt)
        for cond, body in node.elif_branches:
            self.visit(cond)
            for stmt in body:
                self.visit(stmt)
        if node.else_branch:
            for stmt in node.else_branch:
                self.visit(stmt)

    # Visit while loop and analyze condition and body statements
    def visit_WhileLoop(self, node: WhileLoop):
        self.visit(node.condition)
        for stmt in node.body:
            self.visit(stmt)

    # Visit for loop: declare loop variable in new scope and analyze loop body
    def visit_ForLoop(self, node: ForLoop):
        self.visit(node.iterable)
        loop_scope = Scope(parent=self.current_scope)
        if not loop_scope.declare_var(node.var):
            self.error(f"Loop variable '{node.var}' already declared", node)
        prev_scope = self.current_scope
        self.current_scope = loop_scope
        for stmt in node.body:
            self.visit(stmt)
        self.current_scope = prev_scope

    # Visit an expression statement (e.g., standalone function call)
    def visit_ExpressionStatement(self, node: ExpressionStatement):
        self.visit(node.expression)

    # Visit a binary operation node and analyze both left and right operands
    def visit_BinaryOp(self, node: BinaryOp):
        self.visit(node.left)
        self.visit(node.right)
        
        ltype = getattr(node.left, 'inferred_type', None)
        rtype = getattr(node.right, 'inferred_type', None)

        # Handle type inference for numeric operations
        numeric_ops = {'+', '-', '*', '/', '%', '**'}

        if ltype and rtype:
            if node.operator in numeric_ops:
                if ltype in {'int', 'float'} and rtype in {'int', 'float'}:
                    # Type promotion rule: int + float => float
                    if ltype == 'float' or rtype == 'float':
                        node.inferred_type = 'float'
                    else:
                        node.inferred_type = 'int'
                else:
                    self.error(f"Unsupported operand types for {node.operator}: '{ltype}' and '{rtype}'", node)
            elif node.operator in {'==', '!=', '<', '>', '<=', '>='}:
                node.inferred_type = 'bool'
            elif node.operator in {'and', 'or'}:
                if ltype == rtype == 'bool':
                    node.inferred_type = 'bool'
                else:
                    self.error(f"Logical operators require boolean operands, got '{ltype}' and '{rtype}'", node)
            else:
                self.error(f"Unknown binary operator '{node.operator}'", node)

        # Constant folding (optional)
        if hasattr(node.left, 'constant_value') and hasattr(node.right, 'constant_value'):
            try:
                node.constant_value = eval(f"{repr(node.left.constant_value)} {node.operator} {repr(node.right.constant_value)}")
            except Exception:
                pass

            
    # Visit a unary operation node and analyze its operand
    def visit_UnaryOp(self, node: UnaryOp):
        self.visit(node.operand)
        if hasattr(node.operand, 'inferred_type'):
            node.inferred_type = node.operand.inferred_type
        if hasattr(node.operand, 'constant_value'):
            try:
                node.constant_value = eval(f"{node.operator}{node.operand.constant_value}")
            except:
                pass

    # Visit a literal node (e.g., number, string); literals are always valid
    def visit_Literal(self, node: Literal):
        node.inferred_type = type(node.value).__name__
        node.constant_value = node.value


In [23]:
source = """
def square(x):
    return x * x

def add(a, b):
    result = a + b
    return result

value = 5
squared = square(value)
total = add(squared, 10)

def test():
    message = "All good!"
    return message
"""
parseTest= '''
def average_grade(students):  
    total = 0
    for s in students:
        total = total + s
    return total / len(students)

grades = [85, 90, 78]
result = average_grade(grades)
print(result)

'''
semanticTest='''
def printUser(name):
    print("User is " + user)
    
printUser("Hadiah")

'''

lexer = Lexer(semanticTest)
parser = Parser(lexer)
ast = parser.parse()
analyzer = SemanticAnalyzer()

print("AST before semantic analysis:")
print_ast(ast)

try:
    analyzer.analyze(ast)
    print("Semantic analysis passed!\n")
    print("\nSymbol Table:")
    analyzer.global_scope.print_symbols()

    print("\nAST after semantic analysis (with annotations):")
    print_ast(ast)
    # Generate annotated AST graph
    dot = visualize_ast(ast)
    dot.render('ast_tree', view=True, format='png')
except Exception as e:
    print("Semantic analysis failed:", e)
    




AST before semantic analysis:
Program
  FunctionDef(printUser)
    ExpressionStatement
      Call
        Identifier(print)
          BinaryOp(+)
            Literal('User is ')
            Identifier(user)
  ExpressionStatement
    Call
      Identifier(printUser)
        Literal('Hadiah')
Semantic analysis failed: [Line 3, Column 1] Error at 'Call': Call to undefined function 'print'
[Line 3, Column 1] Error at 'Call': Call to undefined function 'print'


In [10]:
from colorama import Fore, Style, init
from IPython.display import Markdown, display
# Enable ANSI color in terminal
init(autoreset=True)

source = """
def add(a, b):
    result = a + b
    return result

x = 1
y = 2.5
z = add(x, y)

"""


lexer = Lexer(source)
parser = Parser(lexer)
ast = parser.parse()

display(Markdown("### AST Before Semantic Analysis"))
print_ast(ast)

analyzer = SemanticAnalyzer()

try:
    analyzer.analyze(ast)

    display(Markdown(f"<span style='color:green; font-weight:bold'>✅ Semantic analysis passed!</span>"))

    display(Markdown("### Symbol Table"))
    analyzer.global_scope.print_symbols()
    
    display(Markdown("### AST After Semantic Analysis (With Annotations)"))
    print_ast(ast)

    dot = visualize_ast(ast)
    dot.render('ast_tree', view=True, format='png')

except Exception as e:
    display(Markdown(f"<span style='color:red; font-weight:bold'>❌ Semantic analysis failed:</span> {e}"))


### AST Before Semantic Analysis

Program
  FunctionDef(add)
    Assignment(target=result)
      BinaryOp(+)
        Identifier(a)
        Identifier(b)
    Return
      Identifier(result)
  Assignment(target=x)
    Literal(1)
  Assignment(target=y)
    Literal(2.5)
  Assignment(target=z)
    Call
      Identifier(add)
        Identifier(x)
        Identifier(y)


<span style='color:green; font-weight:bold'>✅ Semantic analysis passed!</span>

### Symbol Table

Scope:
  Variable: x, Type: int
  Variable: y, Type: float
  Variable: z, Type: float
  Function: add(a: int, b: float), Return: float


### AST After Semantic Analysis (With Annotations)

Program
  FunctionDef(add) [return_type=float]
    Assignment(target=result) [type=float]
      BinaryOp(+) [type=float]
        Identifier(a) [type=int]
        Identifier(b) [type=float]
    Return [type=float]
      Identifier(result) [type=float]
  Assignment(target=x) [type=int]
    Literal(1) [type=int, const=1]
  Assignment(target=y) [type=float]
    Literal(2.5) [type=float, const=2.5]
  Assignment(target=z) [type=float]
    Call [type=float]
      Identifier(add)
        Identifier(x) [type=int]
        Identifier(y) [type=float]


### Code Optimization

In [ ]:
from typing import List, Optional
from dataclasses import dataclass

class Optimizer:
    def __init__(self):
        self.optimizations_applied = 0
        self.modified = True  # Track if any optimizations were made
        self.constants = {}   # track constant values
        self.scope_stack = [{}]  # Stack of scopes for tracking variables in different scopes
    
    def visit_Identifier(self, node: Identifier) -> ASTNode:
        """Propagate known constant values"""
        # Look for constant value in all scopes, from innermost to outermost
        for scope in reversed(self.scope_stack):
            if node.name in scope:
                self.modified = True
                self.optimizations_applied += 1
                # Create new literal and preserve any existing attributes
                new_literal = Literal(value=scope[node.name], line=node.line, column=node.column)
                # Copy any additional attributes that might exist (like type annotations)
                for attr in dir(node):
                    if not attr.startswith('_') and attr not in ['name', 'line', 'column']:
                        if hasattr(node, attr) and not callable(getattr(node, attr)):
                            try:
                                setattr(new_literal, attr, getattr(node, attr))
                            except:
                                pass
                return new_literal
        return node
    
    def optimize(self, node: ASTNode) -> ASTNode:
        """Main optimization entry point"""
        if node is None:
            return None
            
        # Keep optimizing until no more changes can be made
        max_iterations = 10  # Prevent infinite loops
        iteration = 0
        
        while self.modified and iteration < max_iterations:
            self.modified = False
            old_optimizations = self.optimizations_applied
            node = self.visit(node)
            iteration += 1
            
            # If no new optimizations were applied, we're done
            if self.optimizations_applied == old_optimizations:
                break
                
        return node
    
    def visit(self, node: ASTNode) -> Optional[ASTNode]:
        """Visit and potentially transform a node"""
        if node is None:
            return None
            
        method = 'visit_' + type(node).__name__
        visitor = getattr(self, method, self.generic_visit)
        return visitor(node)
    
    def generic_visit(self, node: ASTNode) -> ASTNode:
        """Default visitor that traverses all fields"""
        for field in getattr(node, '__dataclass_fields__', {}):
            value = getattr(node, field)
            if isinstance(value, ASTNode):
                optimized = self.visit(value)
                if optimized is not None:
                    setattr(node, field, optimized)
            elif isinstance(value, list):
                new_list = []
                for item in value:
                    if isinstance(item, ASTNode):
                        optimized = self.visit(item)
                        if optimized is not None:
                            new_list.append(optimized)
                    else:
                        new_list.append(item)
                setattr(node, field, new_list)
        return node

    def visit_Program(self, node: Program) -> Program:
        """Optimize the program's body"""
        optimized_body = []
        for stmt in node.body:
            result = self.visit(stmt)
            if result is not None:
                if isinstance(result, list):
                    optimized_body.extend(result)
                else:
                    optimized_body.append(result)
        node.body = optimized_body
        return node

    def visit_BinaryOp(self, node: BinaryOp) -> ASTNode:
        """Enhanced constant folding for binary operations"""
        # Visit operands first to ensure constant propagation happens
        node.left = self.visit(node.left)
        node.right = self.visit(node.right)
        
        # Try to evaluate if both operands are literals
        left_val = getattr(node.left, 'value', None) if isinstance(node.left, Literal) else None
        right_val = getattr(node.right, 'value', None) if isinstance(node.right, Literal) else None
        
        if left_val is not None and right_val is not None:
            try:
                if node.operator == '+':
                    value = left_val + right_val
                elif node.operator == '-':
                    value = left_val - right_val
                elif node.operator == '*':
                    value = left_val * right_val
                elif node.operator == '/':
                    value = left_val / right_val
                elif node.operator == '%':
                    value = left_val % right_val
                elif node.operator == '==':
                    value = left_val == right_val
                elif node.operator == '!=':
                    value = left_val != right_val
                elif node.operator == '<':
                    value = left_val < right_val
                elif node.operator == '>':
                    value = left_val > right_val
                elif node.operator == '<=':
                    value = left_val <= right_val
                elif node.operator == '>=':
                    value = left_val >= right_val
                else:
                    return node
                
                self.modified = True
                self.optimizations_applied += 1
                # Create new literal and preserve any existing attributes
                new_literal = Literal(value=value, line=node.line, column=node.column)
                # Copy any additional attributes that might exist (like type annotations)
                for attr in dir(node):
                    if not attr.startswith('_') and attr not in ['left', 'right', 'operator', 'line', 'column', 'value']:
                        if hasattr(node, attr) and not callable(getattr(node, attr)):
                            try:
                                setattr(new_literal, attr, getattr(node, attr))
                            except:
                                pass
                return new_literal
            except (ZeroDivisionError, TypeError, ValueError):
                # Don't optimize if operation would cause an error
                return node
        return node

    def visit_IfStatement(self, node: IfStatement) -> Optional[List[ASTNode]]:
        """Optimize if statements"""
        # First optimize the condition
        node.condition = self.visit(node.condition)
        
        # If condition is a constant literal
        if isinstance(node.condition, Literal):
            self.modified = True
            self.optimizations_applied += 1
            if node.condition.value:
                # True condition - only keep then branch
                optimized_then = []
                for stmt in node.then_branch:
                    result = self.visit(stmt)
                    if result is not None:
                        if isinstance(result, list):
                            optimized_then.extend(result)
                        else:
                            optimized_then.append(result)
                return optimized_then
            elif node.else_branch:
                # False condition - only keep else branch
                optimized_else = []
                for stmt in node.else_branch:
                    result = self.visit(stmt)
                    if result is not None:
                        if isinstance(result, list):
                            optimized_else.extend(result)
                        else:
                            optimized_else.append(result)
                return optimized_else
            else:
                # False condition with no else - remove entirely
                return []
        
        # Optimize all branches
        node.then_branch = []
        for stmt in node.then_branch:
            result = self.visit(stmt)
            if result is not None:
                if isinstance(result, list):
                    node.then_branch.extend(result)
                else:
                    node.then_branch.append(result)
                    
        node.elif_branches = []
        for cond, body in node.elif_branches:
            optimized_cond = self.visit(cond)
            optimized_body = []
            for stmt in body:
                result = self.visit(stmt)
                if result is not None:
                    if isinstance(result, list):
                        optimized_body.extend(result)
                    else:
                        optimized_body.append(result)
            node.elif_branches.append((optimized_cond, optimized_body))
            
        if node.else_branch:
            optimized_else = []
            for stmt in node.else_branch:
                result = self.visit(stmt)
                if result is not None:
                    if isinstance(result, list):
                        optimized_else.extend(result)
                    else:
                        optimized_else.append(result)
            node.else_branch = optimized_else
        
        return node

    def visit_WhileLoop(self, node: WhileLoop) -> Optional[WhileLoop]:
        """Optimize while loops"""
        node.condition = self.visit(node.condition)
        
        # If condition is a constant false, remove the loop
        if isinstance(node.condition, Literal) and not node.condition.value:
            self.modified = True
            self.optimizations_applied += 1
            return None
            
        # Optimize loop body
        optimized_body = []
        for stmt in node.body:
            result = self.visit(stmt)
            if result is not None:
                if isinstance(result, list):
                    optimized_body.extend(result)
                else:
                    optimized_body.append(result)
        node.body = optimized_body
        return node

    def visit_FunctionDef(self, node: FunctionDef) -> FunctionDef:
        """Handle function scope"""
        self.scope_stack.append({})  # Push new scope
        
        optimized_body = []
        for stmt in node.body:
            result = self.visit(stmt)
            if result is not None:
                if isinstance(result, list):
                    optimized_body.extend(result)
                else:
                    optimized_body.append(result)
        node.body = optimized_body
        
        self.scope_stack.pop()  # Pop function scope
        return node

    def visit_Return(self, node: Return) -> Return:
        """Optimize return value expressions"""
        if node.value:
            node.value = self.visit(node.value)
        return node

    def visit_Assignment(self, node: Assignment) -> Assignment:
        """Optimize assignment values and track constants"""
        # First optimize the value expression
        node.value = self.visit(node.value)
        
        # If the result is a literal, track it as a constant
        if isinstance(node.value, Literal):
            self.scope_stack[-1][node.target] = node.value.value
            # Mark as modified to ensure another pass happens
            self.modified = True
        else:
            # If assigning a non-constant, remove from constants if it exists
            if node.target in self.scope_stack[-1]:
                del self.scope_stack[-1][node.target]
        
        return node

    def visit_ExpressionStatement(self, node: ExpressionStatement) -> ExpressionStatement:
        """Optimize expressions in expression statements"""
        node.expression = self.visit(node.expression)
        return node

    def visit_Call(self, node: Call) -> Call:
        """Optimize function calls (visit arguments)"""
        if hasattr(node, 'args'):
            node.args = [self.visit(arg) for arg in node.args]
        return node

# Example usage:
def optimize_ast(ast: ASTNode) -> ASTNode:
    optimizer = Optimizer()
    optimized_ast = optimizer.optimize(ast)
    print(f"Applied {optimizer.optimizations_applied} optimizations")
    return optimized_ast

### Just for debugging the optimizer cuz of some issues

In [37]:
from typing import List, Optional
from dataclasses import dataclass

class DebugOptimizer:
    def __init__(self):
        self.optimizations_applied = 0
        self.modified = True
        self.constants = {}
        self.scope_stack = [{}]
        self.debug = True  # Enable debug output
    
    def log(self, message):
        if self.debug:
            print(f"[OPTIMIZER DEBUG] {message}")
    
    def visit_Identifier(self, node: Identifier) -> ASTNode:
        """Propagate known constant values"""
        self.log(f"Visiting identifier: {node.name}")
        self.log(f"Current scope stack: {self.scope_stack}")
        
        # Look for constant value in all scopes, from innermost to outermost
        for i, scope in enumerate(reversed(self.scope_stack)):
            if node.name in scope:
                self.log(f"Found constant {node.name} = {scope[node.name]} in scope {len(self.scope_stack)-1-i}")
                self.modified = True
                self.optimizations_applied += 1
                new_literal = Literal(value=scope[node.name], line=node.line, column=node.column)
                
                # Copy any additional attributes from the original node
                for attr in ['type', 'const', 'scope']:
                    if hasattr(node, attr):
                        try:
                            setattr(new_literal, attr, getattr(node, attr))
                            self.log(f"Copied attribute {attr} = {getattr(node, attr)}")
                        except:
                            pass
                            
                return new_literal
        
        self.log(f"No constant found for {node.name}")
        return node
    
    def optimize(self, node: ASTNode) -> ASTNode:
        """Main optimization entry point"""
        if node is None:
            return None
        
        self.log("Starting optimization")
        max_iterations = 10
        iteration = 0
        
        while self.modified and iteration < max_iterations:
            self.log(f"Optimization pass {iteration + 1}")
            self.modified = False
            old_optimizations = self.optimizations_applied
            
            node = self.visit(node)
            iteration += 1
            
            new_optimizations = self.optimizations_applied - old_optimizations
            self.log(f"Pass {iteration} applied {new_optimizations} optimizations")
            
            if new_optimizations == 0:
                self.log("No new optimizations, stopping")
                break
                
        self.log(f"Optimization complete after {iteration} passes")
        return node
    
    def visit(self, node: ASTNode) -> Optional[ASTNode]:
        """Visit and potentially transform a node"""
        if node is None:
            return None
            
        method = 'visit_' + type(node).__name__
        visitor = getattr(self, method, self.generic_visit)
        return visitor(node)
    
    def generic_visit(self, node: ASTNode) -> ASTNode:
        """Default visitor that traverses all fields"""
        for field in getattr(node, '__dataclass_fields__', {}):
            value = getattr(node, field)
            if isinstance(value, ASTNode):
                optimized = self.visit(value)
                if optimized is not None:
                    setattr(node, field, optimized)
            elif isinstance(value, list):
                new_list = []
                for item in value:
                    if isinstance(item, ASTNode):
                        optimized = self.visit(item)
                        if optimized is not None:
                            new_list.append(optimized)
                    else:
                        new_list.append(item)
                setattr(node, field, new_list)
        return node

    def visit_Program(self, node: Program) -> Program:
        """Optimize the program's body"""
        self.log("Visiting Program")
        optimized_body = []
        for stmt in node.body:
            result = self.visit(stmt)
            if result is not None:
                if isinstance(result, list):
                    optimized_body.extend(result)
                else:
                    optimized_body.append(result)
        node.body = optimized_body
        return node

    def visit_BinaryOp(self, node: BinaryOp) -> ASTNode:
        """Enhanced constant folding for binary operations"""
        self.log(f"Visiting BinaryOp: {node.operator}")
        
        # Visit operands first
        node.left = self.visit(node.left)
        node.right = self.visit(node.right)
        
        # Check if we can fold
        left_val = getattr(node.left, 'value', None) if isinstance(node.left, Literal) else None
        right_val = getattr(node.right, 'value', None) if isinstance(node.right, Literal) else None
        
        self.log(f"Left operand: {type(node.left).__name__}, value: {left_val}")
        self.log(f"Right operand: {type(node.right).__name__}, value: {right_val}")
        
        if left_val is not None and right_val is not None:
            try:
                if node.operator == '+':
                    value = left_val + right_val
                elif node.operator == '-':
                    value = left_val - right_val
                elif node.operator == '*':
                    value = left_val * right_val
                elif node.operator == '/':
                    value = left_val / right_val
                elif node.operator == '%':
                    value = left_val % right_val
                else:
                    self.log(f"Unknown operator: {node.operator}")
                    return node
                
                self.log(f"Folding {left_val} {node.operator} {right_val} = {value}")
                self.modified = True
                self.optimizations_applied += 1
                
                new_literal = Literal(value=value, line=node.line, column=node.column)
                
                # Copy attributes from original node
                for attr in ['type', 'const', 'scope']:
                    if hasattr(node, attr):
                        try:
                            setattr(new_literal, attr, getattr(node, attr))
                        except:
                            pass
                            
                return new_literal
            except Exception as e:
                self.log(f"Error during folding: {e}")
                return node
        return node

    def visit_Assignment(self, node: Assignment) -> Assignment:
        """Optimize assignment values and track constants"""
        self.log(f"Visiting Assignment: {node.target}")
        
        # First optimize the value expression
        node.value = self.visit(node.value)
        
        # If the result is a literal, track it as a constant
        if isinstance(node.value, Literal):
            self.log(f"Storing constant: {node.target} = {node.value.value}")
            self.scope_stack[-1][node.target] = node.value.value
            self.modified = True
        else:
            self.log(f"Assignment {node.target} is not a constant: {type(node.value).__name__}")
            # Remove from constants if it exists
            if node.target in self.scope_stack[-1]:
                del self.scope_stack[-1][node.target]
                self.log(f"Removed {node.target} from constants")
        
        return node

    def visit_FunctionDef(self, node: FunctionDef) -> FunctionDef:
        """Handle function scope"""
        self.log(f"Entering function: {getattr(node, 'name', 'unnamed')}")
        self.scope_stack.append({})
        
        optimized_body = []
        for stmt in node.body:
            result = self.visit(stmt)
            if result is not None:
                if isinstance(result, list):
                    optimized_body.extend(result)
                else:
                    optimized_body.append(result)
        node.body = optimized_body
        
        self.scope_stack.pop()
        self.log(f"Exiting function")
        return node

    def visit_Return(self, node: Return) -> Return:
        """Optimize return value expressions"""
        self.log("Visiting Return")
        if node.value:
            node.value = self.visit(node.value)
        return node

    def visit_ExpressionStatement(self, node: ExpressionStatement) -> ExpressionStatement:
        """Optimize expressions in expression statements"""
        node.expression = self.visit(node.expression)
        return node

    def visit_Call(self, node: Call) -> Call:
        """Optimize function calls"""
        if hasattr(node, 'args'):
            node.args = [self.visit(arg) for arg in node.args]
        return node

# Modified test function
def debug_optimize_ast(ast: ASTNode) -> ASTNode:
    optimizer = DebugOptimizer()
    optimized_ast = optimizer.optimize(ast)
    print(f"Applied {optimizer.optimizations_applied} optimizations")
    return optimized_ast

### Testing

In [ ]:
from colorama import Fore, Style, init
from IPython.display import Markdown, display
init(autoreset=True)

def test_optimization(source_code, test_name=""):
    print(f"\n=== Testing Optimization: {test_name} ===")
    print(f"Source code:\n{source_code}")
    
    # Parse and analyze first
    lexer = Lexer(source_code)
    parser = Parser(lexer)
    ast = parser.parse()
    
    # Initialize analyzer with built-ins
    analyzer = SemanticAnalyzer()
    
    try:
        analyzer.analyze(ast)
        
        print("\nBefore optimization:")
        print_ast(ast)
        
        # Run optimization
        optimized_ast = optimize_ast(ast)
        
        print("\nAfter optimization:")
        print_ast(optimized_ast)
        
        # Visualize before/after
        #dot_before = visualize_ast(ast)
        #dot_before.render('ast_before_opt', view=True, format='png')
        
        #dot_after = visualize_ast(optimized_ast)
        #dot_after.render('ast_after_opt', view=True, format='png')
        
    except SemanticError as e:
        print(f"\nSemantic Error: {e}")

# Test cases
# 1. Constant folding
# test_optimization("""
# x = 2 + 3 * 4
# y = 10 + 20 + 30
# """, "Constant Folding")

# # 2. Dead code elimination in if statements
# test_optimization("""
# if True:
#     x = 10
#     y = 20
# else:
#     x = 30
#     y = 40

# if False:
#     a = "This will be removed"
# """, "Dead Code in If Statements")

# 3. Dead code in while loops
# test_optimization("""
# while False:
#     print("This loop will be removed")
    
# x = 1
# while True:
#     x = x + 1
# """, "Dead Code in While Loops")

# 4. Complex expression folding
test_optimization("""
def calculate():
    a = 5 + 3 * 2
    b = 10 / 2
    c = a + b - (2 * 3)
    return c

result = calculate()
""", "Complex Expression Folding")

# 5. Mixed optimizations
# test_optimization("""
# def test(x):
#     y = 2 + 3 * 4
#     if True:
#         return y + 10
#     else:
#         return y - 10

# while False:
#     print("Dead code")

# result = test(5)
# """, "Mixed Optimizations")


=== Testing Optimization: Complex Expression Folding ===
Source code:

def calculate():
    a = 5 + 3 * 2
    b = 10 / 2
    c = a + b - (2 * 3)
    return c

result = calculate()


Before optimization:
Program
  FunctionDef(calculate) [return_type=int]
    Assignment(target=a) [type=int]
      BinaryOp(+) [type=int, const=11]
        Literal(5) [type=int, const=5]
        BinaryOp(*) [type=int, const=6]
          Literal(3) [type=int, const=3]
          Literal(2) [type=int, const=2]
    Assignment(target=b) [type=int]
      BinaryOp(/) [type=int, const=5.0]
        Literal(10) [type=int, const=10]
        Literal(2) [type=int, const=2]
    Assignment(target=c) [type=int]
      BinaryOp(-) [type=int]
        BinaryOp(+) [type=int]
          Identifier(a) [type=int]
          Identifier(b) [type=int]
        BinaryOp(*) [type=int, const=6]
          Literal(2) [type=int, const=2]
          Literal(3) [type=int, const=3]
    Return [type=int]
      Identifier(c) [type=int]
  Assignment